# Day 25: 猜字遊戲

語言模型可以以字詞為單位作N-gram，也可以以字母為單位。為了讓大家對語言模型有個更清楚的理解，我們用英文猜字遊戲Hangman為範例來學習。

## 猜字遊戲Hangman

<a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a>的玩法是透過一個人寫下一個字，讓另一個人來猜。對方一次會猜一個英文字母，猜對的話就把字母寫到正確的格子上，猜錯的話則會記錄下來。若是猜錯的次數達到一個數字，則猜字的一方輸。

這裡我們先開發一個簡易版：

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word是將要被猜的字、guesser是我們之後會陸續寫進來的猜字模型（真人猜字或AI）、max_mistakes是最多可以錯誤的次數、
        verbose為True時表示互動性猜字（AI猜字時可以改False）、guesser_args是一個keyword argument。
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word) # 把要被猜的字轉成暗文
    guessed = set()
    if verbose:
        print("開始猜字遊戲，提示：", ' '.join(mask), '長度為', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("你還有", (max_mistakes-mistakes), "次機會。")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('你猜了：', guess)
        if guess in guessed:
            if verbose:
                print('這個字母已經猜過了！')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('猜對了，', ' '.join(mask))
            else:
                if verbose:
                    print('抱歉，再猜猜')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('恭喜你贏了！')
            return mistakes
        
    if verbose:
        print('沒有機會了，正確字是', secret_word)    
    return mistakes

這裡我們先寫一個人機互動猜字版：

In [2]:
def human(mask, guessed, **kwargs):
    """
    可以手動遊玩
    """
    print('請輸入你要猜的字：')
    return input().lower().strip()

interactive = True

試玩遊戲：

In [3]:
if interactive:
    hangman('algorithm', human, 8, True)

開始猜字遊戲，提示： _ _ _ _ _ _ _ _ _ 長度為 9
你還有 2 次機會。
請輸入你要猜的字：
q
你猜了： q
抱歉，再猜猜
你還有 1 次機會。
請輸入你要猜的字：
r
你猜了： r
猜對了， _ _ _ _ r _ _ _ _
你還有 1 次機會。
請輸入你要猜的字：
i
你猜了： i
猜對了， _ _ _ _ r i _ _ _
你還有 1 次機會。
請輸入你要猜的字：
p
你猜了： p
抱歉，再猜猜
沒有機會了，正確字是 algorithm


## 準備訓練和測試集

開始寫我們的AI猜字之前，要先準備訓練集和測試集。從[這篇文章](http://datagenetics.com/blog/april12012/index.html)中可以看到，照著統計學來猜字是個不錯的做法，因此我們也自己來訓練一組模型，照著出現的頻率為順序進行猜字。

我們使用之前用過的*Brown Corpus*來訓練AI猜字演算法，為了刻意增加猜字的難度，AI在測試時的字和訓練的字會是不一樣的(把訓練集和測試集拆開)，如此才能訓練出更通用的模型。

我們使用 `nltk.corpus.Brown` 裡的 `words()` 方法，從中將非英文字母的文字去除，同時將每個字小寫化。之後，我們用 `numpy.random.shuffle` 來隨機刷取文集中的字，以分成訓練集和測試集。測試集裡有1000字，剩下的字都會到訓練集裡面。

In [4]:
from nltk.corpus import brown
import numpy as np

np.random.seed(12345)

# word_set 儲存Brown Corpus中所有獨特的字型
word_set = []
# test_set 儲存1000個字的測試集
test_set = []
# training_set 將剩下的字存到訓練集中
training_set = []


word_s = set([])

for word in brown.words():
    if word.isalpha():
        word = word.lower()
        if word not in word_s:
            word_s.add(word)

word_set = list(word_s)
np.random.shuffle(word_set)

test_set = word_set[:1000]
training_set = word_set[1000:]

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


這裡一來就不是一個人寫一個字讓對方猜了，而是電腦和人類的對決：

In [5]:
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

開始猜字遊戲，提示： _ _ _ _ _ _ 長度為 6
你還有 2 次機會。
請輸入你要猜的字：
q
你猜了： q
抱歉，再猜猜
你還有 1 次機會。
請輸入你要猜的字：
z
你猜了： z
抱歉，再猜猜
沒有機會了，正確字是 umpire


# Day 26 三種AI猜字方法

## 第一種猜字方法：隨機猜字

為了設下一個基準，我們先設計一種*AI*方法--每次從26個字母中隨機選取一個字母來猜。這裡我先將26個字母存到 `list` 中，再用 `numpy.random.choice` 隨機選取。

`test_guesser` 是用來測試平均猜錯次數的方法。

In [9]:
def test_guesser(guesser, test=test_set):
    """
        這個方法是用來測試平均猜錯的次數。
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [10]:
import string

def random_guesser(mask, guessed, **kwargs):
    """
        隨機猜字
    """

    alphabets = []
    for letter in range(97,123):
        if chr(letter) not in guessed:
            alphabets.append(chr(letter))

    picked = np.random.choice(alphabets)
    return picked

# 若要看看機器是怎麼猜字的，可以把下面這句打開
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("平均錯誤次數：", result)


Average number of incorrect guesses:  16.675


## 第二種猜法：Unigram Guesser

我們可以嘗試用*Unigram*模型來訓練。我們需要知道每個字母的出現頻率，接著照出現頻率的高低來進行猜字。每當猜完一個字之後就應該把猜過的字去掉。

In [11]:
from collections import Counter

# unigram_counts 儲存了整個訓練及中每個字母的出現次數
unigram_counts = Counter()

for word in training_set:
    for letter in word:
        unigram_counts[letter] += 1

print(unigram_counts)


def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        這個方法實作了Unigram Guesser，會根據Unigram Model每次回傳一個要猜的字。
    """
    
    unigram_keys = []

    # 照出現頻率將字母排序
    for i in range(len(unigram_counts)):
        unigram_keys.append(unigram_counts.most_common()[i][0])

    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys:
            unigram_keys.remove(letter)

    return unigram_keys[0]

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("平均猜錯次數：", result)

Counter({'e': 35197, 'i': 26051, 'a': 24350, 's': 23745, 'n': 22604, 'r': 22069, 't': 20629, 'o': 19194, 'l': 16690, 'c': 12609, 'd': 12299, 'u': 10167, 'm': 8695, 'g': 8600, 'p': 8530, 'h': 7334, 'b': 5738, 'y': 5349, 'f': 4216, 'v': 3442, 'k': 2970, 'w': 2786, 'z': 1023, 'x': 836, 'j': 641, 'q': 529})

平均猜錯次數： 10.55


## 第三種猜法：根據文字長度猜字

從和昨天[同一篇文章](http://datagenetics.com/blog/april12012/index.html)中我們看到，不同的文字長度，每個字母出現的頻率不盡相同，例如，短的字比較不會出現前綴或後綴。在這裡，我們針對不同的文字長度設計不一樣的猜字順序。

In [13]:
from collections import defaultdict

# unigram_counts_by_length 將文字長度和字母頻率map在一起
unigram_counts_by_length = defaultdict(Counter)

# 幫每一種文字長度寫不同的Unigram Model
for word in training_set:    
    this_count = Counter()
    for letter in word:
        this_count[letter] += 1
        unigram_counts_by_length[len(word)] += this_count
        this_count = Counter()

        
def exclude_guessed_letters(length_model, guessed):
    unigram_keys_by_length = []
    # 照出現頻率將字母排序
    for i in range(len(unigram_counts_by_length[length_model])):
        unigram_keys_by_length.append(unigram_counts_by_length[length_model].most_common()[i][0])
    
    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys_by_length:
            unigram_keys_by_length.remove(letter)
    
    return unigram_keys_by_length


lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1

print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    
    length_model = len(mask)
    # 若要猜的文字長度不在unigram model時，我們將一長度來猜。
    while length_model not in lengths:
        length_model -= 1
    
    unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    # 若這個文字長度沒有猜字選項了，從附近的文字長度找
    while len(unigram_keys_by_length) == 0:
        if length_model < 20:
            length_model += 1
        else:
            length_model -= 1
        unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    return unigram_keys_by_length[0]


#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("平均猜錯次數：", result)

defaultdict(<class 'collections.Counter'>, {11: Counter({'e': 2931, 'i': 2741, 'n': 2266, 't': 2148, 'a': 2014, 's': 1983, 'r': 1918, 'o': 1707, 'l': 1320, 'c': 1224, 'u': 849, 'd': 798, 'p': 776, 'm': 739, 'g': 634, 'h': 520, 'b': 418, 'y': 413, 'f': 298, 'v': 289, 'k': 134, 'w': 123, 'x': 73, 'z': 69, 'q': 44, 'j': 26}), 6: Counter({'e': 4254, 'a': 2687, 's': 2609, 'r': 2529, 'i': 2036, 'n': 1990, 'o': 1979, 'l': 1951, 't': 1859, 'd': 1636, 'u': 1140, 'c': 1058, 'm': 941, 'g': 883, 'h': 874, 'p': 842, 'b': 784, 'y': 704, 'f': 495, 'k': 451, 'w': 430, 'v': 396, 'z': 125, 'j': 107, 'x': 94, 'q': 50}), 10: Counter({'e': 4325, 'i': 3624, 'n': 3068, 's': 2906, 't': 2792, 'a': 2774, 'r': 2659, 'o': 2377, 'l': 1935, 'c': 1670, 'd': 1398, 'u': 1292, 'g': 1094, 'p': 1071, 'm': 1054, 'h': 821, 'b': 613, 'y': 551, 'f': 460, 'v': 446, 'w': 207, 'k': 176, 'z': 109, 'x': 96, 'q': 67, 'j': 55}), 8: Counter({'e': 5633, 'i': 3905, 's': 3703, 'a': 3682, 'n': 3515, 'r': 3500, 't': 2904, 'o': 2787, 'l':